# Домашнее задание к занятию 2.1: Деревья решений. Классификация

## Обзор прошедшего занятия

Что мы делали в классе:

**Задание 1**
* строили деревья
* критерии информативности которых написали даже сами
* визуализировали границы принятия решений в 2d
* и рисовали сами деревья


**Задание 2**
* приняли участие в соревновании на Kaggle, переварив кучу текстовых фичей в численные, проведя кросс-валидацию и сделав сабмит


**Задание 3**
* построили руками несколько метрик качества бинарной классификации


**Задание 4**
* использовали их для оценки классификации разделения статей Ведомостей по топикам

*дополнительно было много приятных ништяков. Например, облако слов, мультипоточность в целях парсинга, удобный инструмент для нахождения правильной css разметки, сохранение моделей в статичные файлы, разделение строк на слова и лемматизация этих слов*

## Домашнее задание

**Lvl 1:**

* взять подготовленные раннее данные из задачи **Titanic**, обучиться на них с помощью дерева решений и кросс-валидации и сделать сабмит
* кросс-валидацию желательно сделать сразу по нескольким фичам ( параметр *grid* в *GridSearchCV* )
* определить самые важные фичи
* вывести дерево решений (можете попробовать установить pydot и webgraphviz для отрисовки деревьев внутри ноутбука)

**Результат:** скрины нового сабмита на Kaggle и построенного дерева

**Lvl 2:** (опционально)

* С помощью функций sklearn.metrics.auc, precision, recall составить функцию для расчёта ROC-AUC, ROC-PRC
* Придумать себе интересную задачу на основе данных из интернета =) Спарсить ещё какой-нибудь сайт (не Ведомости) и решить задачу классификации. Делать свои проекты - круто. Если будут - кидайте мне =) [@NikitaKuznetsov](http://t.me/NikitaKuznesov)

In [4]:
import pandas as pd
import numpy as np


In [5]:
##
## 1. Получаем выборки - тестовую и валидационную
##

In [6]:
train = pd.read_csv('titanic_train.csv')
test = pd.read_csv('titanic_test.csv')

In [7]:
##
## 2. Смотрим данные
##

In [8]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
for i in train.columns:
    print (i, train[i].unique())


PassengerId [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 24

In [13]:
for i in test.columns:
    print (i, test[i].unique())

PassengerId [ 892  893  894  895  896  897  898  899  900  901  902  903  904  905
  906  907  908  909  910  911  912  913  914  915  916  917  918  919
  920  921  922  923  924  925  926  927  928  929  930  931  932  933
  934  935  936  937  938  939  940  941  942  943  944  945  946  947
  948  949  950  951  952  953  954  955  956  957  958  959  960  961
  962  963  964  965  966  967  968  969  970  971  972  973  974  975
  976  977  978  979  980  981  982  983  984  985  986  987  988  989
  990  991  992  993  994  995  996  997  998  999 1000 1001 1002 1003
 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017
 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031
 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045
 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055 1056 1057 1058 1059
 1060 1061 1062 1063 1064 1065 1066 1067 1068 1069 1070 1071 1072 1073
 1074 1075 1076 1077 1078 1079 1080 1081 1082 1083 1084 1085 1086

In [14]:
##
## 3 Очищаем данные
##

In [15]:
# drop useless columns
useless_columns = ['PassengerId', 'Ticket', 'Cabin', 'Embarked', 'Name']
train.drop(useless_columns, axis=1, inplace=True)
test.drop(useless_columns, axis=1, inplace=True)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [16]:
# remove nan in Age column
for i in range(0, len(train)):
    if np.isnan(train.iloc[i]['Age']):
        train.at[i,'Age'] = 0  

for i in range(0, len(test)):
    if np.isnan(test.iloc[i]['Age']):
        test.at[i,'Age'] = 0  
    

In [17]:
# conver columns to ids
from sklearn.preprocessing import LabelEncoder

In [18]:
le = {}
encoded_cols = [ 'Sex', 'Age']
for i in encoded_cols:
    le[i] = LabelEncoder()
    le[i].fit( pd.concat([train[i],test[i]]) )
    train[i] = le[i].transform(train[i])
    test[i] = le[i].transform(test[i])
    print ( i, len(le[i].classes_) )    

Sex 2
Age 99


In [19]:
# find and remove fucking nan in test
for i in range(0, len(test)):
    for j in test.columns:
        if np.isnan( test.iloc[i][j] ):
            print (i,j)
            test.at[i,j] = 0

152 Fare


In [20]:
y = train['Survived']
train.drop('Survived', axis=1, inplace=True)
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,33,1,0,7.2500
1,1,0,58,1,0,71.2833
2,3,0,40,0,0,7.9250
3,1,0,54,1,0,53.1000
4,3,1,54,0,0,8.0500


In [21]:
##
## Обучаем модель
##

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
clf = DecisionTreeClassifier(max_depth=3)

In [24]:
clf.fit(train, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
clf.feature_importances_

array([0.1658566 , 0.66222252, 0.        , 0.02500432, 0.036241  ,
       0.11067557])

In [26]:
train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [27]:
test.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [28]:
#y_pred_proba = clf.predict_proba(test)
ytest = clf.predict(test)

In [29]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [30]:
get_tree_dot_view(clf, list(test.columns), str(ytest))

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="Sex <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]\nclass = [", fillcolor="#e5813960"] ;
1 [label="Pclass <= 2.5\ngini = 0.383\nsamples = 314\nvalue = [81, 233]\nclass = 0", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="Fare <= 28.856\ngini = 0.1\nsamples = 170\nvalue = [9, 161]\nclass = 0", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="gini = 0.18\nsamples = 70\nvalue = [7, 63]\nclass = 0", fillcolor="#399de5e3"] ;
2 -> 3 ;
4 [label="gini = 0.039\nsamples = 100\nvalue = [2, 98]\nclass = 0", fillcolor="#399de5fa"] ;
2 -> 4 ;
5 [label="Fare <= 23.35\ngini = 0.5\nsamples = 144\nvalue = [72, 72]\nclass = [", fillcolor="#e5813900"] ;
1 -> 5 ;
6 [label="gini = 0.484\nsamples = 117\nvalue = [48, 69]\nclass = 0", fillcolor="#399de54e"] ;
5 -> 6 ;
7 [label="gini = 0.198\nsamples = 27\nvalue = [24, 3]\nclass = [", fillcolor="#e58139df"] ;
5 -> 7 ;
8 [label="Fare <